Let's create the Account table with a check constraint that ensures the balance cannot be negative, and then define the test cases.

In [9]:
CREATE TABLE Account (
    AccountID INT PRIMARY KEY,
    Balance DECIMAL(10, 2) CHECK (Balance >= 0)
);

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'Account' in the database.

Total execution time: 00:00:00.006

Inset some initial data

In [8]:
INSERT into Account (AccountID, Balance) VALUES(1,1000)
INSERT INTO Account (AccountID, Balance) VALUES(2, 1000)

: Msg 2627, Level 14, State 1, Line 1
Violation of PRIMARY KEY constraint 'PK__Account__349DA586ECBA6856'. Cannot insert duplicate key in object 'dbo.Account'. The duplicate key value is (1).

The statement has been terminated.

: Msg 2627, Level 14, State 1, Line 2
Violation of PRIMARY KEY constraint 'PK__Account__349DA586ECBA6856'. Cannot insert duplicate key in object 'dbo.Account'. The duplicate key value is (2).

The statement has been terminated.

Total execution time: 00:00:00.026

Check the update

In [21]:
SELECT * FROM Account

(1 row affected)

(2 rows affected)

Total execution time: 00:00:00.007

AccountID,Balance
1,1000.00
2,1000.00


### **ATOMICITY (All or Nothing)**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">In an atomic transaction, either all operations succeed or none succeed. If any part fails, the entire transaction is rolled back to its original state, and the database remains consistent. This prevents partial updates that could lead to data corruption or inconsistencies.</span>

Let's Test the Atomicity.

The default behaviour in MYSQl is that if an error occurs within a transaction, the specific statement that caused the error is terminated. Still, the transaction itself is not automatically rolled back. This means subsequent statements within the same transaction can still be executed unless you explicitly handle the rollback.

We can use the <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">XACT_ABORT method</span>

When SET XACT\_ABORT is ON, the transaction is terminated and rolled back if a Transact-SQL statement raises a run-time error.

We can achieve this atomicity behaviour in other ways aswell, by using TRY and CATCH

In [24]:
SET XACT_ABORT ON;

BEGIN TRANSACTION

    -- THIS WILL BE SUCCESSFUL AS THE BALANCE WILL STILL BE GREATER THAN 0
    UPDATE ACCOUNT SET BALANCE = BALANCE - 500 WHERE ACCOUNTID = 1

    -- LETS UPDATE THE ACCOUNTID 1 AGAIN AND TRY TO BREACH NON NEGATIVE BALANCE CONSTRAINT
    UPDATE ACCOUNT SET BALANCE = BALANCE - 700 WHERE ACCOUNTID = 1
    -- THIS SHOULD THROW THE ERROR

    --NOW LETS UPDATE ACCOUNTID 2 AGAIN
    UPDATE Account SET Balance = Balance + 500.00 WHERE AccountID = 2;

COMMIT TRANSACTION

(1 row affected)

: Msg 547, Level 16, State 0, Line 9
The UPDATE statement conflicted with the CHECK constraint "CK__Account__Balance__25A691D2". The conflict occurred in database "master", table "dbo.Account", column 'Balance'.

Total execution time: 00:00:00.025

In [25]:
SELECT * from Account

(2 rows affected)

Total execution time: 00:00:00.009

AccountID,Balance
1,1000.00
2,1000.00


Lets try the same change again with XACT\_ABORT OFF

In [26]:
SET XACT_ABORT OFF;

BEGIN TRANSACTION

    -- THIS WILL BE SUCCESSFUL AS THE BALANCE WILL STILL BE GREATER THAN 0
    UPDATE ACCOUNT SET BALANCE = BALANCE - 500 WHERE ACCOUNTID = 1

    -- LETS UPDATE THE ACCOUNTID 1 AGAIN AND TRY TO BREACH NON NEGATIVE BALANCE CONSTRAINT
    UPDATE ACCOUNT SET BALANCE = BALANCE - 700 WHERE ACCOUNTID = 1
    -- THIS SHOULD THROW THE ERROR

    --NOW LETS UPDATE ACCOUNTID 2 AGAIN
    UPDATE Account SET Balance = Balance + 500.00 WHERE AccountID = 2;

COMMIT TRANSACTION

(1 row affected)

: Msg 547, Level 16, State 0, Line 9
The UPDATE statement conflicted with the CHECK constraint "CK__Account__Balance__25A691D2". The conflict occurred in database "master", table "dbo.Account", column 'Balance'.

The statement has been terminated.

(1 row affected)

Total execution time: 00:00:00.026

In [27]:
SELECT * FROM ACCOUNT

(2 rows affected)

Total execution time: 00:00:00.015

AccountID,Balance
1,500.00
2,1500.00
